In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
import math
from imblearn.over_sampling import SMOTE

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
#Read the data into dataframe
car_df = pd.read_csv('newCardata.csv')
car_features = pd.read_csv('finalDataPreprocess.csv')

#encode the label as 1's or 0's
label_Number = LabelEncoder()
car_df['FraudFound'] = label_Number.fit_transform(car_df['FraudFound'].astype('str'))
car_label = car_df['FraudFound']
print(car_features.shape)

(15419, 97)


In [3]:
#divide the data into train and test set

X_train,X_test,y_train,y_test = train_test_split(car_features,car_label,random_state=3,test_size=0.25)
print(X_train.shape,X_test.shape)


(11564, 97) (3855, 97)


In [4]:
#build the model
clf= RandomForestClassifier(n_estimators=100) 

#train the model
model= clf.fit(X_train,y_train)

#test the model and find model's performance metric
predicted= model.predict(X_test)

# calculating specifity and sensitivity
# 0  := Negative(FraudNotFound)
# 1 := Positive (FraudFound)
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()

print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)


Confusion Matrix:
 [[3605    4]
 [ 242    4]]
TN: 3605
FP: 4
FN: 242
TP: 4
Accuracy: 93.6186770428
Sensitivity: 1.62601626016
Specificity: 99.889165974


In [5]:
#With SMOTE 

#apply the Smote
sm= SMOTE()
features,labels= sm.fit_sample(car_features,car_label)
print(features.shape,labels.shape)

#split the smote generated data into the train and test set
X_train,X_test,y_train,y_test= train_test_split(features,labels,random_state=3,test_size=0.25)
print(X_train.shape,X_test.shape)



(28992, 97) (28992,)
(21744, 97) (7248, 97)


In [6]:
#build the model
clf= RandomForestClassifier(n_estimators=100) 

#train the model
model= clf.fit(X_train,y_train)

#test the model and find model's performance metric
predicted= model.predict(X_test)

# calculating specifity and sensitivity
# 0  := Negative(FraudNotFound)
# 1 := Positive (FraudFound)
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()

print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3567   56]
 [ 149 3476]]
TN: 3567
FP: 56
FN: 149
TP: 3476
Accuracy: 97.1716335541
Sensitivity: 95.8896551724
Specificity: 98.4543196246


In [10]:
#apply the Smote
sm= SMOTE()
features,labels= sm.fit_sample(car_features,car_label)
print(features.shape,labels.shape)

#split the smote generated data into the train and test set
X_train,X_test,y_train,y_test= train_test_split(features,labels,random_state=3,test_size=0.25)
print(X_train.shape,X_test.shape)



(28992, 97) (28992,)
(21744, 97) (7248, 97)


In [11]:
#find the total number of the fraudulent claims in the smote generated data
#find the all the fradulent claims in the dataset
total_fraud= 0
total_Nonfraud= 0
print("Total no of data in smote generated data: ",labels.shape[0])
for i in range(labels.shape[0]):
    if labels[i]==1:
        total_fraud += 1
    else:
        total_Nonfraud += 1
print("Total fraud in smote generated data: ", total_fraud)
print("Total Non-fraud in smote generated data: ", total_Nonfraud)

Total no of data in smote generated data:  28992
Total fraud in smote generated data:  14496
Total Non-fraud in smote generated data:  14496


In [13]:
#testing only on the original test data
from numpy import linalg as LA

#print the original size of the smote generated test set
print( "Original X_test  and y_test", (X_test.shape, y_test.shape))

#find the all the fradulent claims in the dataset
indexOFfraudulent= []
columnfeatures= list(car_features.columns.values)
fraudulentClaims= pd.DataFrame(columns=columnfeatures)
j= 0
for i in range(car_label.shape[0]):
    if car_label[i]==1:
        fraudulentClaims.loc[j]= car_features.loc[i]
        j += 1
print("Total number of fraudulent claims in original data:",fraudulentClaims.shape[0])
#print(y_test[1000])

Original X_test  and y_test ((7248, 97), (7248,))
Total number of fraudulent claims in original data: 923


In [23]:

#remove the smote generated sample from the test set.
k= 0
index= []


#make empty dataframes for storing the new test data
new_X_test= pd.DataFrame(columns=columnfeatures)
new_y_test= pd.DataFrame(columns=['label'])

#convert numpy array to pd.DataFrame
X_test= pd.DataFrame(X_test)


for i in range(X_test.shape[0]):
#for i in range(1000):
    element= X_test.loc[i]
    if (int(y_test[i])==1):
        #print("Inside condition y_test== 1, y_test[i] is:",y_test[i])
        for j in range(fraudulentClaims.shape[0]):
            data = pd.Series(element).values
            fraudClaims= fraudulentClaims.loc[j]
            fraudClaims= pd.Series(fraudClaims).values
            euclidean_distance= LA.norm(data-fraudClaims,2)   #calculate the euclidean distance

            if (euclidean_distance == 0):
                index.append(i)
                new_X_test.loc[k]= [element[elm] for elm in range(element.shape[0])]
                #print("After euclidean distance==0, y_test[i] is:",y_test[i])
                new_y_test.loc[k]= y_test[i]
                break
    else:
        new_X_test.loc[k]= [element[elm] for elm in range(element.shape[0])]
        #print("Else condition, y_test[i]:",y_test[i])
        new_y_test.loc[k]= y_test[i]
    k += 1


In [17]:
print("New X_test and y_test shape:",new_X_test.shape,new_y_test.shape)


New X_test and y_test shape: (3855, 97) (3855, 1)


In [18]:
#find the original test set from smote generated dataset
print(new_X_test.shape)
print(new_y_test.shape)

#change the dataframe into numpy array
mod_X_test= new_X_test.values
mod_y_test= new_y_test.values

fraud= 0
for i in range(mod_y_test.shape[0]):
    if(mod_y_test[i]==1):
        fraud += 1
print("Total Number of fraud in modified test set:",fraud)



(3855, 97)
(3855, 1)
Total Number of fraud in modified test set: 232


In [21]:
#train the model with new modified test set
#build the model
clf= RandomForestClassifier(n_estimators=100) 

#train the model
model= clf.fit(X_train,y_train)

#test the model and find model's performance metric
#predicted= model.predict(mod_X_test)
predicted= model.predict(mod_X_test)

# calculating specifity and sensitivity
# 0  := Negative(FraudNotFound)
# 1 := Positive (FraudFound)
#cm = confusion_matrix(mod_y_test,predicted)
cm = confusion_matrix(mod_y_test,predicted)

print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()

print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3566   57]
 [ 140   92]]
TN: 3566
FP: 57
FN: 140
TP: 92
Accuracy: 94.8897535668
Sensitivity: 39.6551724138
Specificity: 98.4267181893
